In [ ]:
!pip install datasets 
!pip install transformers
!pip install librosa
!pip install jiwer
!pip install huggingface_hub==0.1

заходим на хаги ваги, чтобы пушить в репку

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs

качаем датасет мазилы на ру языке

In [ ]:
from datasets import load_dataset, load_metric
train_ds = load_dataset("mozilla-foundation/common_voice_9_0", "ru", split="train+validation+other", use_auth_token=True)
test_ds = load_dataset("mozilla-foundation/common_voice_9_0", "ru", split="test", use_auth_token=True)

In [ ]:
train_ds[0]

Уберем ненужные поля

In [ ]:
train_ds = train_ds.remove_columns(['client_id','up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'])
test_ds = test_ds.remove_columns(['client_id','up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'])

Почистим тексты от ignore символов

In [ ]:
import re
CHARS_TO_IGNORE = [",", "?", "¿", ".", "!", "¡", ";", "；", ":", '""', "%", '"', "�", "ʿ", "·", "჻", "~", "՞",
                   "؟", "،", "।", "॥", "«", "»", "„", "“", "”", "「", "」", "‘", "’", "《", "》", "(", ")", "[", "]",
                   "{", "}", "=", "`", "_", "+", "<", ">", "…", "–", "°", "´", "ʾ", "‹", "›", "©", "®", "—", "→", "。",
                   "、", "﹂", "﹁", "‧", "～", "﹏", "，", "｛", "｝", "（", "）", "［", "］", "【", "】", "‥", "〽",
                   "『", "』", "〝", "〟", "⟨", "⟩", "〜", "：", "！", "？", "♪", "؛", "/", "\\", "º", "−", "^", "ʻ", "ˆ"]

chars_to_remove_regex = f"[{re.escape(''.join(CHARS_TO_IGNORE))}]"

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    batch["sentence"] = re.sub('[-]', ' ', batch["sentence"]).lower()
    return batch         

train_ds = train_ds.map(remove_special_characters)
test_ds = test_ds.map(remove_special_characters)          

Соберем словарь из всех сиволов текстов



In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = train_ds.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_ds.column_names)
vocab_test = test_ds.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_ds.column_names)

vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_list

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

засейвим полученный словарь

In [ ]:
import json
with open('vocab.json', 'w') as f:
    json.dump(vocab_dict, f)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Запушим в репу наш токенайзер

In [ ]:
repo = 'tekila_sunset_lame_ru_wave2vec'

In [ ]:
tokenizer.push_to_hub(repo)

Создадим экстрактор и объединим с токенайзером в процессоре

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
feature_extractor.push_to_hub(repo)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.push_to_hub(repo)

Скастим колонку в Audio

In [ ]:
import torchaudio
from datasets import Audio
train_ds = train_ds.cast_column("audio", Audio(sampling_rate=16_000))
test_ds = test_ds.cast_column("audio", Audio(sampling_rate=16_000))

Отфильтруем датасет, если не хватает памяти (её никогда не хватает)

In [ ]:
import librosa
def is_audio_in_length_range(batch):
    audio = batch["audio"]
    if len(re.split(r'\W+',batch["sentence"]))>100:
      return False
    return False if librosa.get_duration(y=audio["array"],sr=audio["sampling_rate"]) > 3 else True

In [ ]:
len(train_ds)

50099

In [ ]:
train_ds = train_ds.filter(
            is_audio_in_length_range,
        )

In [ ]:
len(train_ds)

7107

Подготовим входные данные

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
train_ds = train_ds.map(prepare_dataset, remove_columns=train_ds.column_names)
test_ds = test_ds.map(prepare_dataset, remove_columns=test_ds.column_names)

Скопипастим коллатор из гайдика как тру прогеры



In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import AutoProcessor


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.AutoProcessor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: AutoProcessor
    padding: Union[bool, str] = "longest"
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

докинем метрику wer


In [ ]:
import numpy as np
wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Создадим объект модели

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    attention_dropout=0.094,
    hidden_dropout=0.047,
    feat_proj_dropout=0.04,
    mask_time_prob=0.082,
    layerdrop=0.041,
    activation_dropout=0.055,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)
# model.to('cuda')

In [ ]:
model.freeze_feature_extractor()

Зададим параметры длдя тренировки

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo,
  group_by_length=True,
  per_device_train_batch_size=10,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=10,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=50,
  learning_rate=5e-5,
  warmup_steps=500,
  save_total_limit=10,
  push_to_hub=True,
  load_best_model_at_end=True,
  greater_is_better=False,
  report_to="tensorboard",
  metric_for_best_model='eval_wer',
)

Чекнем видяху

In [ ]:
!nvidia-smi -L 

Загрузим в трейнер и стартанем тренировку

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=processor.tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()

function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);

ЕТО В КОНСОЛЬ, ЧТОБЫ НЕ ОТКИСЛО